In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException

# from pynput.keyboard import Key, Controller

import time

from re import sub
from decimal import Decimal


event_link = 'https://www.cheersapp.com.br/evento/15-anos-da-aaacf-7022' # o final
# event_link = 'https://www.cheersapp.com.br/evento/moulin-rouge-6984' # pra teste (tem primeiro lote)

chatNames = ["saved messages","Karys"]
messages = ["⚠⚠⚠ ATENÇÃO ⚠⚠⚠\nVendas do rolê liberadas!!", "Corra!"]


In [ ]:
options = Options()
options.add_argument("user-data-dir=C:\\global\\dadoswebdriver")
options.binary_location='C:\\Users\\Artur Zahn\\AppData\\Local\\Vivaldi\\Application\\vivaldi.exe'

nav = webdriver.Chrome(options=options)

In [ ]:
# telegram automations

def enterTelegram():
    nav.get("https://web.telegram.org")


def openChat(txtSearch):
    searchBox = WebDriverWait(nav, timeout=10).until(lambda d: d.find_element(By.CSS_SELECTOR, '.input-search-input'))
    # searchBox = WebDriverWait(nav, timeout=10).until(lambda d: d.find_element(By.CSS_SELECTOR, '.input-field-input'))
    # searchBox = WebDriverWait(nav, timeout=10).until(lambda d: d.find_element(By.CSS_SELECTOR, '.input-search'))
    searchBox.click()
    searchBox.send_keys(txtSearch)
    time.sleep(2)
    
    contactChats = WebDriverWait(nav, timeout=10).until(lambda d: d.find_elements(By.CSS_SELECTOR, '.search-group-contacts .chatlist-chat'))
    selectedchat = contactChats[0]
    selectedchat.click()
    time.sleep(2)

def sendMessage(msg):
    chatInput = WebDriverWait(nav, timeout=10).until(lambda d: d.find_element(By.CSS_SELECTOR, '.input-message-input'))

    for subMsg in msg.split("\n"):
        # print(f"@{subMsg}@")
        chatInput.send_keys(subMsg)
        chatInput.send_keys(Keys.SHIFT, Keys.ENTER)

    # time.sleep(2)
    chatInput.send_keys(Keys.ENTER)

def sendMessages(msgs):
    for msg in msgs:
        sendMessage(msg)

def sendAlertTelegram():
    enterTelegram()
    for chatName in chatNames:
        openChat(chatName)
        sendMessages(messages)
    

In [ ]:
# cheers automation

def goToSelecedEventPage():
    nav.get(event_link)

# check if the current page is an valid event based on the title of the event
def checkIfIsEventPage():
    try:
        title = WebDriverWait(nav, timeout=10).until(lambda d: d.find_element(By.CSS_SELECTOR, '.title-evento-single'))
        print(f'found title in the page')
    except NoSuchElementException:
        print(f'did not found title in the page')
        # print('nao achou')
        return False
    
    return True

def findLots():
    try:
        lotsIndisp = nav.find_elements(By.CSS_SELECTOR, '.event-lote-indiposnivel-box')
    except NoSuchElementException:
        lotsIndisp = []
        
    try:
        lotsDisp = nav.find_elements(By.CSS_SELECTOR, '.event-lote-box')
    except NoSuchElementException:
        lotsDisp = []
    

    qtdTotal = len(lotsIndisp) + len(lotsDisp)

    if qtdTotal <= 0:
        print("found 0 lots (is this going to print sometime??)")
        return False
    
    return {"qtdTotal": qtdTotal, "disp": lotsDisp, "indisp": lotsIndisp}

def findCheapestLot():
    # finding cheapest lot (dont know if its necessary, but anyway...)

    lots = findLots()
    print(f'achou {len(lots["disp"])}/{len(lots["indisp"])}')

    if not lots:
        print("did not found lots")
        return False

    if len(lots["disp"]) <= 0:
        print("did not found available lots")
        return False
    
    
    cheapestLot = None
    cheapestPrice = None

    for lotsDisp in lots['disp']:
        price = lotsDisp.find_element(By.CSS_SELECTOR, '.txt-single-evento-preco-lote')
        txtPrice = price.text
        # print(txtPrice)
        
        # check if read correctly
        if txtPrice[0:3] == 'R$ ':
            txtPrice = txtPrice[3:].replace(",", ".")
            floatPrice = float(txtPrice)
            
            if cheapestLot is None:
                cheapestLot = lotsDisp
                cheapestPrice = floatPrice

            elif cheapestPrice > floatPrice:
                cheapestLot = lotsDisp
                cheapestPrice = floatPrice

    # found cheapest lot

    print(f"cheapest: R$ {cheapestPrice}")

    return cheapestLot

# def getLotButtons(lot):
#     buttons = WebDriverWait(lot, timeout=10).until(lambda d: d.find_elements(By.CSS_SELECTOR, 'button'))
#     # print(len(buttons))
#     return buttons[0].click, buttons[1].click

def getLotButtons(lot):
    # time.sleep(1)
    buttons = WebDriverWait(lot, timeout=5).until(lambda d: d.find_elements(By.CSS_SELECTOR, 'button'))
    print(f"found {len(buttons)} buttons")
    # buttons = lot.find_elements(By.CSS_SELECTOR, 'button')
    # print(len(buttons))
    return buttons[0].click, buttons[1].click

def finishOrder():
    
    bottom = nav.find_element(By.CSS_SELECTOR, ".navbar.fixed-bottom")
    bottom.find_element(By.CSS_SELECTOR, "button").click()

    confirmButton = WebDriverWait(nav, timeout=10).until(lambda d: d.find_element(By.CSS_SELECTOR, '.ra-confirm'))
    confirmButton.click()

    finalizarCompra = WebDriverWait(nav, timeout=10).until(lambda d: d.find_element(By.CSS_SELECTOR, '.btn-carrinho-finalizar'))
    nav.execute_script("arguments[0].scrollIntoView();", finalizarCompra)
    time.sleep(3) #click scroll
    finalizarCompra.click()

    proximo = WebDriverWait(nav, timeout=10).until(lambda d: d.find_element(By.CSS_SELECTOR, '.btn-coupon-buy'))
    nav.execute_script("arguments[0].scrollIntoView();", proximo)
    time.sleep(3) #click scroll
    proximo.click()

    WebDriverWait(nav, timeout=10).until(lambda d: d.find_element(By.CSS_SELECTOR, '.infos-compra')) # esperar pagina carregar
    time.sleep(5)
    proximo2 = WebDriverWait(nav, timeout=10).until(lambda d: d.find_element(By.CSS_SELECTOR, '.btn-coupon-buy'))
    nav.execute_script("arguments[0].scrollIntoView();", proximo2)
    time.sleep(3) #click scroll
    proximo2.click()




def startWatchingCheersEvent():
    global event_link
    
    comprar = True

    while(comprar):

        goToSelecedEventPage()
        print(checkIfIsEventPage())
        
        cheapest = findCheapestLot()

        if cheapest:
            print("found something to buy")
            comprar = False
            try:
                removeOne,addOne = getLotButtons(cheapest)
                nav.execute_script("arguments[0].scrollIntoView();", cheapest)
                time.sleep(1)
                addOne()
                finishOrder()
            except Exception as e:
                comprar = True
                
            time.sleep(5)

            sendAlertTelegram()
        else:
            time.sleep(5)

In [ ]:
# deixar zoom em 40%

startWatchingCheersEvent()